<a href="https://colab.research.google.com/github/itsmuditt/Stock_Price_Prediction/blob/main/News_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **WEB SCRAPING**

---
We will scrape headlines from websites using the piece of code in this segment.


---




In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [ ]:
import datetime
start_date = datetime.datetime(2013, 1, 1)

date_list = [start_date.date() + datetime.timedelta(days=x) for x in range(44976-41275)]
dates = [x.strftime("%d-%m-%Y") for x in date_list]


### **Economic Times Website's Archieves**

---


```
# Doing it for Economic Times website's ARCHIEVES
```



In [ ]:
news = []
for starttime in range(41275, 44976):
  url=f'https://economictimes.indiatimes.com/archivelist/starttime-{starttime}.cms'
  response = requests.get(url)

  soup = BeautifulSoup(response.text, 'html.parser')
  headlines = soup.body.find_all('td', class_="contentbox5")
  day = []
  for x in headlines:
    y = x.find_all('a')
    for heads in y:
      temp = heads.text.strip()
      if len(temp.split())>4:
        day.append(temp)

  news.append(day)

In [ ]:
news_et_df = pd.DataFrame(list(zip(dates, news )))

In [ ]:
news_et_df.to_csv('news_et.csv')


### **Times of India Website's Archieves**

---


```
# Doing it for Times of India website's ARCHIEVES
```



In [ ]:
news_toi = []
for starttime in range(41275, 44976):
  url=f'https://timesofindia.indiatimes.com/archivelist/starttime-{starttime}.cms'
  response = requests.get(url)

  soup = BeautifulSoup(response.text, 'html.parser')
  headlines = soup.body.find_all('span')
  day = []
  for x in headlines:
    y = x.find_all('a')
    for heads in y:
      temp = heads.text.strip()
      if len(temp.split())>4:
        day.append(temp)

  news_toi.append(day)

In [ ]:
news_toi_df = pd.DataFrame(list(zip(dates, news_toi)))

In [ ]:
news_toi_df.to_csv('news_toi.csv')



---



---



## **Revising our DATASET**

---

```
The dataset we were left with, after web scraping, had all the headlines into one column.

The below segment of code, changes it to having as many columns as the particular row requires.

```



---



In [ ]:
import pandas as pd

In [ ]:
net = pd.read_csv('news_et.csv')

In [ ]:
ntoi = pd.read_csv('news_toi.csv')

In [ ]:
def split_sentences(row):
    max_sentences = len(row['Headlines'])
    cnt = 1
    i = 2
    while i < max_sentences:
        stri = ['\', "', '", \'', '\', \'', '", "']
        tmp = [row['Headlines'].find(stri[0], i), row['Headlines'].find(stri[1], i), row['Headlines'].find(stri[2], i), row['Headlines'].find(stri[3], i)]
        ans = 1e10
        for t in tmp:
            if t == -1:
                t = 1e10
            ans = min(ans, t)
        if ans == 1e10:
            ans = max_sentences - 2
        j = ans

        hea = row['Headlines'][i:j]
        i = j + 4
        if hea == '[' or hea == ']':
            continue
        row[f'{cnt}'] = hea
        cnt = cnt + 1
        print(hea)
    return row



---

### **Applying for NET Dataset**


---



In [ ]:
# Apply the function to split sentences
net = net.apply(split_sentences, axis=1)

In [ ]:
# Drop the original 'Headlines' column
net.drop(columns=['Headlines'], inplace=True)

In [ ]:
# Rename the 'old_name' column to 'new_name'
net = net.rename(columns={'Date': '0'})

In [ ]:
# Convert column names to integers and sort them numerically
net.columns = net.columns.astype(int)
net = net.reindex(sorted(net.columns), axis=1)

In [ ]:
net.to_csv('et_rev.csv')



---

### **Applying for NTOI Dataset**


---



In [ ]:
# Apply the function to split sentences
ntoi = ntoi.apply(split_sentences, axis=1)

In [ ]:
# Drop the original 'Headlines' column
ntoi.drop(columns=['Headlines'], inplace=True)

In [ ]:
# Rename the 'old_name' column to 'new_name'
ntoi = ntoi.rename(columns={'Date': '0'})

In [ ]:
# Convert column names to integers and sort them numerically
ntoi.columns = ntoi.columns.astype(int)
ntoi = ntoi.reindex(sorted(ntoi.columns), axis=1)

In [ ]:
ntoi.to_csv('toi_rev.csv')


---

---

---

---


## **Refining DATASET**



---

`
The Dataset has been revised. Every Headline is now in a different cell.
`

---

```
In this segment of code, we refine the dataset.

We keep only :-
> Finance
> Business
> Science
> Economy
as the types of headlines in our dataset.

```



---





### Creating Database to fine-tune our Model

DATASE FROM HUFFPOST[KAGGLE DATASET]

In [5]:
import pandas as pd

In [6]:
new_df = pd.read_json('News_Category_Dataset_v3.json', lines=True)

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# # shuffle the DataFrame rows
# new_df = new_df.sample(frac = 1)

In [ ]:
# dicti = new_df.headline[new_df.category=='QUEER VOICES'].values.tolist()
# print(dicti)

In [8]:
# drop all rows that are redundant
new_df = new_df.drop(new_df[new_df['category'] == 'GOOD NEWS'].index)

In [9]:
# drop all rows that are redundant
new_df = new_df.drop(new_df[new_df['category'] == 'PARENTS'].index)

In [10]:
# drop all rows that are redundant
new_df = new_df.drop(new_df[new_df['category'] == 'WEIRD NEWS'].index)

In [11]:
# drop all rows that are redundant
new_df = new_df.drop(new_df[new_df['category'] == 'QUEER VOICES'].index)

In [12]:
# drop all rows that are redundant
new_df = new_df.drop(new_df[new_df['category'] == 'FIFTY'].index)

In [ ]:
new_df.shape

(193649, 6)

In [13]:
import numpy as np
new_df.index = np.arange(0, len(new_df))

In [17]:
# dct={'MONEY':'FINANCE'}
for j in range(len(new_df)):
  i = new_df.category[j]
  if i == 'ARTS' or i == 'CULTURE & ARTS':
    new_df.category[j] = 'ARTS & CULTURE'
  elif i == 'DIVORCE' or i == 'IMPACT' or i == 'LATINO VOICES' or i == 'PARENTING':
    new_df.category[j] = 'SOCIETY'
  elif i == 'WEDDINGS' or i == 'BLACK VOICES' or i == 'WOMEN':
    new_df.category[j] = 'SOCIETY'
  elif i == 'COLLEGE':
    new_df.category[j] = 'EDUCATION'
  elif i == 'COMEDY' or i == 'STYLE & BEAUTY':
    new_df.category[j] = 'ENTERTAINMENT'
  elif i == 'FOOD & DRINK' or i == 'TASTE':
    new_df.category[j] = 'FOOD'
  elif i == 'GREEN':
    new_df.category[j] = 'ENVIRONMENT'
  elif i == 'HEALTHY LIVING' or i == 'WELLNESS':
    new_df.category[j] = 'HEALTH & MEDICINE'
  elif i == 'HOME & LIVING' or i == 'STYLE' or i == 'TRAVEL':
    new_df.category[j] = 'LIFESTYLE'
  elif i == 'RELIGION':
    new_df.category[j] = 'FAITH'
  elif i == 'SCIENCE' or i == 'TECH':
    new_df.category[j] = 'SCIENCE & TECHNOLOGY'
  elif i == 'THE WORLDPOST' or i == 'U.S. NEWS' or i == 'WORLDPOST':
    new_df.category[j] = 'WORLD NEWS'
  elif i == 'MONEY':
    new_df.category[j] = 'FINANCE'

  # Alternative

  # new_df.category[j]=dct[new_df.category[j]]

<ipython-input-17-ba6f45805496>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.category[j] = 'SOCIETY'
<ipython-input-17-ba6f45805496>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.category[j] = 'LIFESTYLE'
<ipython-input-17-ba6f45805496>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.category[j] = 'ENTERTAINMENT'
<ipython-input-17-ba6f45805496>:19: SettingWithCopyWarning: 
A value is trying to be set on a

In [ ]:
len(set(new_df.category))

17

In [18]:
# drop all rows that are redundant
new_df = new_df.drop(new_df[new_df['category'] == 'FAITH'].index)

# drop all rows that are redundant
new_df = new_df.drop(new_df[new_df['category'] == 'ARTS & CULTURE'].index)

In [ ]:
set(new_df.category)

{'BUSINESS',
 'CRIME',
 'EDUCATION',
 'ENTERTAINMENT',
 'ENVIRONMENT',
 'FINANCE',
 'FOOD',
 'HEALTH & MEDICINE',
 'LIFESTYLE',
 'MEDIA',
 'POLITICS',
 'SCIENCE & TECHNOLOGY',
 'SOCIETY',
 'SPORTS',
 'WORLD NEWS'}

In [19]:
import numpy as np
new_df.index = np.arange(0, len(new_df))

In [ ]:
len(new_df)

187150

DATASET from newsapi client

In [20]:
!pip install newsapi-python

In [21]:
from newsapi.newsapi_client import NewsApiClient

In [22]:
# use your API key here
newsapi = NewsApiClient(api_key='3a50489ee52b4bb8a3729ce2ec9dedf3')

In [23]:
fin_buzz_articles = newsapi.get_everything(q='trading, business and finance', language='en', page_size=100, exclude_domains='https://removed.com')
crypto = newsapi.get_everything(q='cryptocurrency', language='en', page_size=100, exclude_domains='https://removed.com')
stock_market = newsapi.get_everything(q='indian stock market', language='en', page_size=100, exclude_domains='https://removed.com')
# politics = newsapi.get_everything(q='indian politics', language='en', page_size=100, exclude_domains='https://removed.com')
# geopolitics = newsapi.get_everything(q='geopolitics', language='en', page_size=100, exclude_domains='https://removed.com')
# fashion = newsapi.get_everything(q='fashion in india', language='en', page_size=100, exclude_domains='https://removed.com')
# food = newsapi.get_everything(q='food', language='en', page_size=100, exclude_domains='https://removed.com')
# entertainment = newsapi.get_everything(q='entertainment', language='en', page_size=100, exclude_domains='https://removed.com')
# science = newsapi.get_everything(q='science', language='en', page_size=100, exclude_domains='https://removed.com')
# sports = newsapi.get_everything(q='sports', language='en', page_size=100, exclude_domains='https://removed.com')
# health = newsapi.get_everything(q='health and medicine', language='en', page_size=100, exclude_domains='https://removed.com')
# cinema = newsapi.get_everything(q='indian cinema', language='en', page_size=100, exclude_domains='https://removed.com')
# technology = newsapi.get_everything(q='technology, software technology', language='en', page_size=100, exclude_domains='https://removed.com')
# travel = newsapi.get_everything(q='travel', language='en', page_size=100, exclude_domains='https://removed.com')

In [24]:
# technology = pd.DataFrame(technology['articles'])
# technology['category'] = 'Science and Technology'

# science = pd.DataFrame(science['articles'])
# science['category'] = 'Science and Technology'

# entertainment = pd.DataFrame(entertainment['articles'])
# entertainment['category'] = 'Entertainment'

# cinema = pd.DataFrame(cinema['articles'])
# cinema['category'] = 'Entertainment'

# fashion = pd.DataFrame(fashion['articles'])
# fashion['category'] = 'Entertainment'

fin_buzz = pd.DataFrame(fin_buzz_articles['articles'])
fin_buzz['category'] = 'FINANCE'

stocks = pd.DataFrame(stock_market['articles'])
stocks['category'] = 'FINANCE'

crypto = pd.DataFrame(crypto['articles'])
crypto['category'] = 'FINANCE'

# sports = pd.DataFrame(sports['articles'])
# sports['category'] = 'Sports'

# politics = pd.DataFrame(politics['articles'])
# politics['category'] = 'Politics'

# geopolitics = pd.DataFrame(geopolitics['articles'])
# geopolitics['category'] = 'Politics'

# travel = pd.DataFrame(travel['articles'])
# travel['category'] = 'Travel'

# food = pd.DataFrame(food['articles'])
# food['category'] = 'Food'

# health = pd.DataFrame(health['articles'])
# health['category'] = 'Health'



In [25]:
categories = [fin_buzz, stocks, crypto]
df = pd.concat(categories)

In [26]:
# shuffle the DataFrame rows
df = df.sample(frac = 1)

In [27]:
# drop all rows that contain 'Coca Cola'
df = df.drop(df[df['title'] == '[Removed]'].index)

In [ ]:
df.shape

(294, 9)

In [28]:
import numpy as np
df.index = np.arange(0, len(df))

In [29]:
df.rename(columns = {'title':'headline'}, inplace = True)

In [ ]:
new_df.shape

(187150, 6)

In [30]:
df = df.drop('content', axis=1)
df = df.drop('url', axis=1)
df = df.drop('urlToImage', axis=1)
df = df.drop('publishedAt', axis=1)
df = df.drop('source', axis=1)

Now we have 2 datasets, 'df' and 'new_df'

> We will take the headlines and category column from both the datasets and concatenate them

In [31]:
text = new_df['headline'].values
labels = new_df['category'].values

In [32]:
text = np.append(text, df['headline'].values, axis=0)

In [34]:
len(text)

187444

In [33]:
labels = np.append(labels, df['category'].values, axis=0)

In [35]:
len(set(labels))

15

In [36]:
final_df = pd.DataFrame(text, columns=['headline'])

In [37]:
final_df['labels'] = labels

In [38]:
#mapping dict for labels
label_dict = {}
index=0
for l in set(labels):
    label_dict[l] = index
    index += 1

In [ ]:
label_dict

{'HEALTH & MEDICINE': 0,
 'ENTERTAINMENT': 1,
 'FINANCE': 2,
 'SCIENCE & TECHNOLOGY': 3,
 'WORLD NEWS': 4,
 'EDUCATION': 5,
 'SOCIETY': 6,
 'POLITICS': 7,
 'ENVIRONMENT': 8,
 'BUSINESS': 9,
 'MEDIA': 10,
 'FOOD': 11,
 'CRIME': 12,
 'LIFESTYLE': 13,
 'SPORTS': 14}

In [39]:
# shuffle the DataFrame rows
final_df = final_df.sample(frac = 1)

In [40]:
final_df['labels'] = final_df['labels'].map(label_dict)

In [ ]:
final_df.shape

(187444, 2)

In [ ]:
# We have the required data now.

In [41]:
data_texts = final_df["headline"].to_list() # Features (not-tokenized yet)
data_labels = final_df["labels"].to_list() # Lables

### Splitting the DATASET into TRAINING and TEST data

In [42]:
from sklearn.model_selection import train_test_split

In [43]:
def get_train_vali_test(X, y, ratio_train = 0.8, ratio_val = 0.1, ratio_test = 0.1):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ratio_test, random_state=42, shuffle=True)
    ratio_remaining = 1 - ratio_test
    ratio_val_adjusted = ratio_val / ratio_remaining
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=ratio_val_adjusted, random_state=42, shuffle=True)
    return X_train, X_test, X_val, y_train, y_test, y_val

In [44]:
train_texts, test_texts, val_texts, train_labels, test_labels, val_labels = get_train_vali_test(data_texts, data_labels)



```
# DATASET has be split into TRAINING, VALIDATION and TESTING DATA
```



In [45]:
# Installing Transformers with sentencepiece
!pip install --no-cache-dir transformers sentencepiece

In [46]:
from transformers import DistilBertTokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [ ]:
!pip install tf-models-official

In [48]:
import tensorflow as tf
import tensorflow_models as tfm
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [49]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))

## Logit Model

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize
from sklearn.linear_model import LogisticRegression

In [51]:
tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=True,
                        preprocessor=None,  # applied preprocessor in Data Cleaning
                        tokenizer=word_tokenize,
                        use_idf=True,
                        norm='l2',
                        smooth_idf=True,
                        stop_words= 'english',
                        max_df=0.5,
                        sublinear_tf=True)

In [52]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [53]:
X =  tfidf.fit_transform(train_texts)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [54]:
clf = LogisticRegression(max_iter=100000000, C=5).fit(X, train_labels)

In [55]:
X_val = tfidf.transform(val_texts)
X_test = tfidf.transform(test_texts)

In [56]:
from transformers import TFDistilBertForSequenceClassification
from sklearn.metrics import f1_score
from tqdm import tqdm
import pickle
from transformers import BertTokenizer, BertForSequenceClassification
from tabulate import tabulate
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
from tqdm import trange

In [57]:
f1_score(val_labels, clf.predict(X_val), average='micro')

0.6835956255001334

In [58]:
f1_score(test_labels, clf.predict(X_test), average='micro')

0.6885036009602561

### Load Pretrained Model (distilbert)

In [59]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=42)

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

### MODEL Training

In [ ]:
# first training
history = model.fit(train_dataset.shuffle(100).batch(100), epochs=7, batch_size=100,
          validation_data=val_dataset.shuffle(100).batch(100))

In [62]:
#second training
history_second = model.fit(train_dataset.shuffle(110).batch(90), epochs=2, batch_size=90,
          validation_data=val_dataset.shuffle(121).batch(90))

Epoch 1/2
1667/1667 [==============================] - 1734s 1s/step - loss: 0.1022 - accuracy: 0.9666 - val_loss: 1.4192 - val_accuracy: 0.7276
Epoch 2/2
1667/1667 [==============================] - 1734s 1s/step - loss: 0.0865 - accuracy: 0.9712 - val_loss: 1.5070 - val_accuracy: 0.7252


In [90]:
his_dict = history.history
history_second_dict = history_second.history

## Saving model and History

In [65]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [66]:
model.push_to_hub("news_classification_model")
tokenizer.push_to_hub("news_classification_model")

tf_model.h5:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/darkdwine/news_classification_model/commit/d80f6668e8c3eb5e7d716b02afef0cef10567321', commit_message='Upload tokenizer', commit_description='', oid='d80f6668e8c3eb5e7d716b02afef0cef10567321', pr_url=None, pr_revision=None, pr_num=None)

In [67]:
save_directory = "news_classification_model"
model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

('news_classification_model/tokenizer_config.json',
 'news_classification_model/special_tokens_map.json',
 'news_classification_model/vocab.txt',
 'news_classification_model/added_tokens.json')

In [68]:
load_directory = "news_classification_model"
loaded_tokenizer = DistilBertTokenizer.from_pretrained(load_directory)
loaded_model = TFDistilBertForSequenceClassification.from_pretrained(load_directory)

Some layers from the model checkpoint at news_classification_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at news_classification_model and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [69]:
import json

In [85]:
with open("hist_1.json", "w") as outfile:
    json.dump(his_dict, outfile)

In [86]:
with open("hist_2.json", "w") as outfile:
    json.dump(history_second_dict, outfile)

In [91]:
df_hisory_1 = pd.read_json('hist_1.json')
df_hisory_2 = pd.read_json('hist_2.json')

df_history = pd.concat([df_hisory_1, df_hisory_2]).reset_index(drop=True)

In [94]:
df_history.columns

Index(['loss', 'accuracy', 'val_loss', 'val_accuracy', 'epoch'], dtype='object')

In [95]:
df_history.plot(title='training history', y=['accuracy', 'val_accuracy'], xlabel='epoch', ylabel='accuracy')

In [100]:
df_history.plot(title='training history', y=['loss', 'val_loss'], xlabel='epoch', ylabel='loss')

<Axes: title={'center': 'training history'}, xlabel='epoch', ylabel='loss'>

In [80]:
def make_prediciton(input_text):
    predict_input = loaded_tokenizer.encode(input_text,
                                     truncation=True,
                                     padding=True,
                                     return_tensors="tf")

    output = loaded_model(predict_input)[0]

    prediction_value = tf.argmax(output, axis=1).numpy()[0]
    pred_label = [d for d in label_dict if label_dict[d] == prediction_value][0]
    return pred_label

In [81]:
def ev_pred(input_text):
    predict_input = loaded_tokenizer.encode(input_text,
                                     truncation=True,
                                     padding=True,
                                     return_tensors="tf")

    output = loaded_model(predict_input)[0]

    prediction_value = tf.argmax(output, axis=1).numpy()[0]
    return prediction_value

In [82]:
make_prediciton('Look of the Week: Hunter Schafers ethereal Oscars after-party look')

'ENTERTAINMENT'

In [83]:
pred = [ev_pred(t) for t in tqdm(test_texts)]
f1_score(test_labels, pred, average='micro')

100%|██████████| 18745/18745 [39:32<00:00,  7.90it/s]


0.731555081355028



---





---



---

